## Mục Đích Phân Cụm và Câu Hỏi Nghiên Cứu

**Mục tiêu chính:** Xác định và phân tích các phân khúc sản phẩm trên Amazon dựa trên mối quan hệ giữa các yếu tố: đặc tính/tính năng mô tả của sản phẩm, giá bán và đánh giá từ người dùng. Qua đó, tìm hiểu xem các sản phẩm đang được định vị như thế nào trên thị trường về mặt "giá trị" và "chất lượng".

**Tại sao chọn các thuộc tính này?**

* **`about_product` (Mô tả sản phẩm - Dữ liệu Văn bản):**
    * **Đóng góp:** Cung cấp thông tin chi tiết về các tính năng, lợi ích, công dụng, và đặc điểm kỹ thuật của sản phẩm. Đây là yếu tố then chốt để hiểu được "giá trị nội tại" hoặc "chất lượng được quảng bá" của sản phẩm.
    * **Vai trò trong phân cụm:** Giúp nhóm các sản phẩm có bộ tính năng tương tự hoặc hướng đến cùng một nhu cầu sử dụng.

* **`discounted_price` / `actual_price` (Giá sản phẩm - Dữ liệu Số):**
    * **Đóng góp:** Phản ánh chi phí mà người tiêu dùng phải bỏ ra để sở hữu sản phẩm.
    * **Vai trò trong phân cụm:** Là một trục quan trọng để so sánh "giá trị nhận được" so với "chi phí bỏ ra".

* **`rating` (Đánh giá trung bình - Dữ liệu Số):**
    * **Đóng góp:** Thể hiện mức độ hài lòng của khách hàng đã sử dụng sản phẩm, một thước đo quan trọng về "chất lượng cảm nhận" hoặc "chất lượng thực tế" sau khi trải nghiệm.
    * **Vai trò trong phân cụm:** Giúp xác định mức độ chấp nhận và yêu thích của thị trường đối với sản phẩm.

**Các câu hỏi mà việc phân cụm với các thuộc tính này nhằm trả lời:**

1.  **Phân khúc sản phẩm theo giá trị và chất lượng:**
    * Có thể xác định được các nhóm sản phẩm nào mang lại **"giá trị tốt nhất"** (ví dụ: nhiều tính năng được mô tả, rating cao nhưng giá cả phải chăng) không?
    * Có thể phân biệt được nhóm sản phẩm **"cao cấp"** (ví dụ: mô tả nhiều tính năng vượt trội, rating rất cao, và giá cao tương xứng) với nhóm sản phẩm **"cơ bản/giá rẻ"** (ví dụ: mô tả ít tính năng, rating ở mức chấp nhận được, giá thấp) không?

2.  **Phát hiện các trường hợp đặc biệt về định giá và cảm nhận chất lượng:**
    * Có nhóm sản phẩm nào dường như được **"định giá quá cao"** (ví dụ: mô tả nhiều tính năng nhưng rating thực tế thấp so với mức giá) không?
    * Có thể tìm ra những **"món hời tiềm ẩn"** (ví dụ: rating tốt, giá thấp, nhưng có thể phần mô tả chưa làm nổi bật hết các tính năng giá trị) không?

3.  **Hiểu cấu trúc thị trường:**
    * Các cụm sản phẩm hình thành có tương quan như thế nào giữa ba yếu tố: tính năng mô tả, giá cả, và đánh giá của người dùng? (Ví dụ: Một cụm có thể là "các sản phẩm điện tử với mô tả nhiều tính năng, giá cao và rating rất tốt", trong khi một cụm khác là "phụ kiện giá rẻ, mô tả đơn giản, rating ở mức khá").

Bằng cách phân cụm dựa trên sự kết hợp của ba thuộc tính này, chúng ta kỳ vọng sẽ khám phá ra những phân khúc sản phẩm có ý nghĩa, từ đó cung cấp cái nhìn sâu sắc hơn về chiến lược sản phẩm và định vị giá trên thị trường.

In [1]:
import pandas as pd
import numpy as np
import re

In [6]:
df = pd.read_csv("data/amazon.csv")

# 1. MÔ TẢ TẬP DỮ LIỆU 

### 📦 Thông tin về tập Amazon Sales Dataset
#### 🛍️ Thông tin về sản phẩm

| Trường              | Mô tả                                                                 |
|---------------------|----------------------------------------------------------------------|
| `product_id`        | Mã định danh duy nhất của sản phẩm.                                  |
| `product_name`      | Tên sản phẩm.                                                        |
| `category`          | Danh mục sản phẩm.                                                   |
| `discounted_price`  | Giá sau khi giảm (hiển thị với khách hàng).                          |
| `actual_price`      | Giá gốc của sản phẩm trước khi giảm giá.                             |
| `discount_percentage` | Phần trăm giảm giá được tính dựa trên `actual_price`.              |
| `rating`            | Điểm đánh giá trung bình từ người dùng (thang điểm 5).               |
| `rating_count`      | Số lượng người đã đánh giá sản phẩm.                                 |
| `about_product`     | Mô tả ngắn hoặc thông tin kỹ thuật về sản phẩm.                      |
| `img_link`          | Link hình ảnh đại diện của sản phẩm.                                 |
| `product_link`      | Link tới trang chính thức của sản phẩm trên website (Amazon hoặc khác). |

#### 🧑 Thông tin đánh giá từ người dùng

| Trường           | Mô tả                                                                 |
|------------------|----------------------------------------------------------------------|
| `user_id`        | Mã định danh của người dùng đã đánh giá sản phẩm.                    |
| `user_name`      | Tên người dùng đã viết đánh giá. Có thể là nickname hoặc tên thật.   |
| `review_id`      | Mã định danh của đánh giá, dùng để phân biệt từng bài review.        |
| `review_title`   | Tiêu đề ngắn gọn của đánh giá, thể hiện ý chính.                     |
| `review_content` | Nội dung chi tiết của bài đánh giá từ người dùng.                    |

# 2. ĐÁNH GIÁ TẬP DỮ LIỆU 

In [5]:
print("Năm dòng đầu tiên của DataFrame thô:")
df.head()

Năm dòng đầu tiên của DataFrame thô:


,product_id,product_name,category,discounted_price,actual_price,discount_percentage,rating,rating_count,about_product,user_id,user_name,review_id,review_title,review_content,img_link,product_link
0,B07JW9H4J1,Wayona Nylon Braided USB to Lightning Fast Cha...,Computers&Accessories|Accessories&Peripherals|...,₹399,"₹1,099",64%,4.2,"24,269",High Compatibility : Compatible With iPhone 12...,"AG3D6O4STAQKAY2UVGEUV46KN35Q,AHMY5CWJMMK5BJRBB...","Manav,Adarsh gupta,Sundeep,S.Sayeed Ahmed,jasp...","R3HXWT0LRP0NMF,R2AJM3LFTLZHFO,R6AQJGUP6P86,R1K...","Satisfied,Charging is really fast,Value for mo...",Looks durable Charging is fine tooNo complains...,https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Wayona-Braided-WN3LG1-Sy...
1,B098NS6PVG,Ambrane Unbreakable 60W / 3A Fast Charging 1.5...,Computers&Accessories|Accessories&Peripherals|...,₹199,₹349,43%,4.0,"43,994","Compatible with all Type C enabled devices, be...","AECPFYFQVRUWC3KGNLJIOREFP5LQ,AGYYVPDD7YG7FYNBX...","ArdKn,Nirbhay kumar,Sagar Viswanathan,Asp,Plac...","RGIQEG07R9HS2,R1SMWZQ86XIN8U,R2J3Y1WL29GWDE,RY...","A Good Braided Cable for Your Type C Device,Go...",I ordered this cable to connect my phone to An...,https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Ambrane-Unbreakable-Char...
2,B096MSW6CT,Sounce Fast Phone Charging Cable & Data Sync U...,Computers&Accessories|Accessories&Peripherals|...,₹199,"₹1,899",90%,3.9,"7,928",【 Fast Charger& Data Sync】-With built-in safet...,"AGU3BBQ2V2DDAMOAKGFAWDDQ6QHA,AESFLDV2PT363T2AQ...","Kunal,Himanshu,viswanath,sai niharka,saqib mal...","R3J3EQQ9TZI5ZJ,R3E7WBGK7ID0KV,RWU79XKQ6I1QF,R2...","Good speed for earlier versions,Good Product,W...","Not quite durable and sturdy,https://m.media-a...",https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Sounce-iPhone-Charging-C...
3,B08HDJ86NZ,boAt Deuce USB 300 2 in 1 Type-C & Micro USB S...,Computers&Accessories|Accessories&Peripherals|...,₹329,₹699,53%,4.2,"94,363",The boAt Deuce USB 300 2 in 1 cable is compati...,"AEWAZDZZJLQUYVOVGBEUKSLXHQ5A,AG5HTSFRRE6NL3M5S...","Omkar dhale,JD,HEMALATHA,Ajwadh a.,amar singh ...","R3EEUZKKK9J36I,R3HJVYCLYOY554,REDECAZ7AMPQC,R1...","Good product,Good one,Nice,Really nice product...","Good product,long wire,Charges good,Nice,I bou...",https://m.media-amazon.com/images/I/41V5FtEWPk...,https://www.amazon.in/Deuce-300-Resistant-Tang...
4,B08CF3B7N1,Portronics Konnect L 1.2M Fast Charging 3A 8 P...,Computers&Accessories|Accessories&Peripherals|...,₹154,₹399,61%,4.2,"16,905",[CHARGE & SYNC FUNCTION]- This cable comes wit...,"AE3Q6KSUK5P75D5HFYHCRAOLODSA,AFUGIFH5ZAFXRDSZH...","rahuls6099,Swasat Borah,Ajay Wadke,Pranali,RVK...","R1BP4L2HH9TFUP,R16PVJEXKV6QZS,R2UPDB81N66T4P,R...","As good as original,Decent,Good one for second...","Bought this instead of original apple, does th...",https://m.media-amazon.com/images/W/WEBP_40237...,https://www.amazon.in/Portronics-Konnect-POR-1...


In [7]:
print(f"\nKích thước của DataFrame thô: {df.shape[0]} dòng, {df.shape[1]} cột")


Kích thước của DataFrame thô: 1465 dòng, 16 cột


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1465 entries, 0 to 1464
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   product_id           1465 non-null   object
 1   product_name         1465 non-null   object
 2   category             1465 non-null   object
 3   discounted_price     1465 non-null   object
 4   actual_price         1465 non-null   object
 5   discount_percentage  1465 non-null   object
 6   rating               1465 non-null   object
 7   rating_count         1463 non-null   object
 8   about_product        1465 non-null   object
 9   user_id              1465 non-null   object
 10  user_name            1465 non-null   object
 11  review_id            1465 non-null   object
 12  review_title         1465 non-null   object
 13  review_content       1465 non-null   object
 14  img_link             1465 non-null   object
 15  product_link         1465 non-null   object
dtypes: obj

Ta thấy dữ liệu gồm có 1465 dòng với 16 cột dữ liệu dưới dạng object.

Các cột discounted_price, actual_price, discount_percentage, rating, rating_count cần chuyển về  dạng số:
| **Cột**              | **Kiểu mới** | **Ghi chú**                                                            |
|----------------------|--------------|------------------------------------------------------------------------|
| `discounted_price`   | `float`      | Cần loại bỏ ký tự `₹`, dấu phẩy nếu có trước khi chuyển đổi kiểu      |
| `actual_price`       | `float`      | Cần loại bỏ ký tự `₹`, dấu phẩy nếu có trước khi chuyển đổi kiểu      |
| `rating`             | `float`      | Đã ở dạng số thập phân, chỉ cần ép kiểu nếu cần                        |
| `discount_percentage`| `int`        | Cần loại bỏ dấu `%` trước khi ép kiểu                                 |
| `rating_count`       | `int`        | Loại bỏ dấu phẩy hàng nghìn (`,`) trước khi ép kiểu                   |

Các cột product_name, category, about_product, review_title, review_content cần xử lý dạng văn bản, rút gọn nội dung, tên sản phẩm/ danh mục: 


| **Cột**             | **Cách xử lý**                                                                                                                                                                  |
|---------------------|---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| `product_name`      | Rút gọn tên sản phẩm, loại bỏ các ký tự đặc biệt                                                                                 |
| `category`          | Đây là chuỗi phân cấp danh mục, ví dụ: `Computers&Accessories\|Accessories&Peripherals\|...\|USBCables`. Có thể xử lý theo các cách:                                          |
|                     | - Trích xuất danh mục **cụ thể nhất** (ví dụ: `USBCables`)                                                                                                                      |
|                     | - Kết hợp danh mục **chính và cuối cùng** (ví dụ: `Computers&Accessories - USBCables`)                                                                                          |
|                     | - Hoặc tách thành **các cột riêng biệt** theo từng cấp phân loại (ví dụ: `cat_lvl_1`, `cat_lvl_2`, ...)                                                                        |
| `about_product`     | Rút gọn nội dung mô tả hoặc áp dụng các kỹ thuật xử lý ngôn ngữ tự nhiên (NLP): loại bỏ từ dừng, chuẩn hoá văn bản, tóm tắt nội dung.                                          |
| `review_title`      | Làm sạch dấu câu, loại bỏ trùng lặp, chuẩn hoá chữ thường/hoa.                                  |
| `review_content`    | Tương tự như `review_title`: làm sạch, chuẩn hoá, có thể dùng `tóm tắt`, `TF-IDF`, `Word2Vec` |

In [16]:
print("\nGiá trị null ở các cột:")
df.isnull().sum()


Giá trị null ở các cột:


product_id             0
product_name           0
category               0
discounted_price       0
actual_price           0
discount_percentage    0
rating                 0
rating_count           2
about_product          0
user_id                0
user_name              0
review_id              0
review_title           0
review_content         0
img_link               0
product_link           0
dtype: int64

In [22]:
# Phần trăm giá trị null ở các cột
print("\nPhần trăm giá trị null ở các cột:")
print(df.isnull().mean().round(4) * 100)


Phần trăm giá trị null ở các cột:
product_id             0.00
product_name           0.00
category               0.00
discounted_price       0.00
actual_price           0.00
discount_percentage    0.00
rating                 0.00
rating_count           0.14
about_product          0.00
user_id                0.00
user_name              0.00
review_id              0.00
review_title           0.00
review_content         0.00
img_link               0.00
product_link           0.00
dtype: float64


Có cột rating_count có tỷ lệ % Null là 0.14%, có thể xóa/bỏ đi được. 


# 3. TIỀN XỬ LÝ DỮ LIỆU 

## 3.1. Chuẩn hóa định dạng các cột 

## 3.2. Làm sạch dữ liệu 

# 4. KHÁM PHÁ VÀ PHÂN TÍCH DỮ LIỆU (EDA)

# 5. LOẠI BỎ OUTLIER, CHUẨN HÓA VĂN BẢN 